In [1]:
from algo_reasoning.src.models.network import EncodeProcessDecode
from algo_reasoning.src.data import CLRSDataset, CLRSSampler, collate
from torch.utils.data import DataLoader
import torch
from algo_reasoning.src.data import CLRSData

2024-09-09 10:23:56.722246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 10:23:56.802759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 10:23:56.803253: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 10:23:58.405069: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ronald/Documents/env/algo_reasoning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autono

## Load Dataset and Model

In [2]:
algos = [
    'articulation_points',
    'activity_selector',
    'bellman_ford',
    'bfs',
    'binary_search',
    'bubble_sort',
    'dag_shortest_paths',
    'dfs',
    'dijkstra',
    'find_maximum_subarray_kadane',
    'floyd_warshall',
    'graham_scan',
    'heapsort',
    'insertion_sort',
    'jarvis_march',
    'kmp_matcher',
    'matrix_chain_order',
    'minimum',
    'mst_kruskal',
    'mst_prim',
    'naive_string_matcher',
    'optimal_bst',
    'quickselect',
    'quicksort',
    'segments_intersect',
    'strongly_connected_components',
    'task_scheduling',
    'topological_sort',
]

In [3]:
algorithms = ["three_kinds_dice"]
three_kinds_dice_ds = CLRSDataset(algorithms, "train", "tmp/CLRS30")
three_kinds_dice_sampler = CLRSSampler(three_kinds_dice_ds, algorithms, 32, replacement=False)

three_kinds_dice_loader = DataLoader(three_kinds_dice_ds, batch_sampler=three_kinds_dice_sampler, collate_fn=collate)

In [4]:
algorithms2 = ["jarvis_march"]
jarvis_march_ds = CLRSDataset(algorithms2, "train", "tmp/CLRS30")
jarvis_march_sampler = CLRSSampler(jarvis_march_ds, algorithms2, 32, replacement=False)

jarvis_march_loader = DataLoader(jarvis_march_ds, batch_sampler=jarvis_march_sampler, collate_fn=collate)

In [5]:
processor = EncodeProcessDecode(algos).processor
state_dict = torch.load("checkpoints/Generalist_GMPNN_WithoutTeacherForcing/Generalist_GMPNN_WithoutTeacherForcing-epoch=95-val_loss=0.39.ckpt", map_location=torch.device("cuda"))["state_dict"]
new_state_dict = {}
for key in state_dict:
    if "processor" in key:
        new_state_dict[key.replace("model.processor.", "")] = state_dict[key]

processor.load_state_dict(new_state_dict)

generalist_model = EncodeProcessDecode(algos, pretrained_processor=processor)

state_dict = torch.load("checkpoints/ThreeKindsDice_Pretrained/ThreeKindsDice_Pretrained-epoch=92-val_loss=0.02.ckpt", map_location=torch.device("cuda"))["state_dict"]
new_state_dict = {}
for key in state_dict:
    if "model" in key:
        new_state_dict[key.replace("model.", "")] = state_dict[key]
three_kinds_dice_model = EncodeProcessDecode(algorithms)
three_kinds_dice_model.load_state_dict(new_state_dict)

/tmp/ipykernel_6374/2852374136.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("checkpoints/Generalist_GMPNN_WithoutTeacherForcing/Generalist_GMP

<All keys matched successfully>

In [6]:
with torch.no_grad():
    generalist_model.eval()
    three_kinds_dice_model.eval()

    batch = next(iter(jarvis_march_loader))
    output, hidden_embeddings = generalist_model(batch)

    batch.hints = CLRSData()
    batch.outputs = CLRSData()
    batch.max_length = torch.tensor(1)
    output2, hidden_embeddings2 = generalist_model(batch)

/home/ronald/Documents/algo_reasoning/algo_reasoning/src/data.py:199: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f"{self.data_folder}/{algorithm}/{self.

In [7]:
sample = jarvis_march_ds[0]
output, hidden_embeddings = generalist_model(sample)

In [8]:
hidden_embedding_three_kinds_dice = []
hidden_embedding_generalist = []

for i in range(len(three_kinds_dice_ds)):
    sample = three_kinds_dice_ds[i]
    output, hidden_embeddings = three_kinds_dice_model(sample)
    hidden_embedding_three_kinds_dice.append(hidden_embeddings)

    jarvis_march_input = CLRSData(inputs=CLRSData(pos=sample.inputs.pos, x=sample.hints.score_D1[:, 0, :], y=sample.hints.score_D2[:, 0, :]), 
                                  hints=CLRSData(), 
                                  length=torch.tensor(1).float(), 
                                  max_length=torch.tensor(1).float(),
                                  outputs=CLRSData(), 
                                  algorithm="jarvis_march")
    
    output_jarvis_march, hidden_embeddings_jarvis_march = generalist_model(jarvis_march_input)
    hidden_embedding_generalist.append(hidden_embeddings_jarvis_march)

In [11]:


hidden_embedding_three_kinds_dice[0].shape

torch.Size([2, 16, 128])